# ClusterAI 2020
# Ciencia de Datos - Ingeniería Industrial - UTN BA
# clase_XX: Clasifación con Redes Recurrentes
### Elaborado por: Nicolás Aguirre

### Montar G.Drive


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

A continuacion copiamos los archivos de nuestro G.Drive al directorio de la PC virtual

Esto es recomendable para bases de datos grandes, no tanto para este caos en particular, donde los archivos pesan pocos Mb.

En otras cirunstancias acceder a videos o imagenes, puede pasa a ser un cuello de botella, ya que la informacion de nuestro G.Drive no esta fisicamente en la misma ubicacion que la notebook.

De esta manera, accedemos a los archivos de forma mas rapida.

In [ ]:
!cp /content/gdrive/My\ Drive/clusterai_2020/clases/datasets/clase12/*.txt /content/

# IMPORT

In [ ]:
#Datos
import pandas as pd
import numpy as np
#Graficos 
import matplotlib.pyplot as plt
import seaborn as sns

# Experimental Scenario
---
Data sets consists of multiple multivariate time series. Each data set is further divided into training and test subsets. Each time series is from a different engine – i.e., the data can be considered to be from a fleet of engines of the same type. Each engine starts with different degrees of initial wear and manufacturing variation which is unknown to the user. This wear and variation is considered normal, i.e., it is not considered a fault condition. There are three operational settings that have a substantial effect on engine performance. These settings are also included in the data. The data is contaminated with sensor noise.

The engine is operating normally at the start of each time series, and develops a fault at some point during the series. In the training set, the fault grows in magnitude until system failure. In the test set, the time series ends some time prior to system failure. The objective of the competition is to predict the number of remaining operational cycles before failure in the test set, i.e., the number of operational cycles after the last cycle that the engine will continue to operate. Also provided a vector of true Remaining Useful Life (RUL) values for the test data.

The data are provided as a zip-compressed text file with 26 columns of numbers, separated by spaces. Each row is a snapshot of data taken during a single operational cycle, each column is a different variable.

# Carga de los Datos

Vamos a cargar los datos con Pandas, y a nombrar las columnas para que sea mas facil el manejo.

In [ ]:
#Nombres de las columnas
columns_name = ['Unit','Time','OS1','OS2','OS3',
                's01','s02','s03','s04','s05','s06','s07','s08','s09','s10',
                's11','s12','s13','s14','s15','s16','s17','s18','s19','s20',
                's21','s22','s23','Name']

features_columns = ['OS1','OS2','OS3',
                's01','s02','s03','s04','s05','s06','s07','s08','s09','s10',
                's11','s12','s13','s14','s15','s16','s17','s18','s19','s20',
                's21']
#Nombre de los archivos
names = ['FD001','FD002','FD003','FD004']

# Definimos los train y test df
train_df = pd.DataFrame(columns=columns_name)
test_df = pd.DataFrame(columns=columns_name)

In [ ]:
for i_name in names:
    # Aqui cargamos los .txt en dataframes de pivot
    # que luego los pasaremos al dataframe definitivo
    df_train_pivot = pd.read_csv('train_'+i_name+'.txt', sep=" ", header=None)
    df_test_pivot = pd.read_csv('test_'+i_name+'.txt', sep=" ", header=None)
    # Creamos la columna Name y guardamos el nombre dle archivo
    df_train_pivot['Name'] = i_name
    df_test_pivot['Name'] = i_name

    #Nombramos las columnas segun el readme.txt
    df_train_pivot.columns = columns_name
    df_test_pivot.columns = columns_name
    
    #Juntamos el df-pivot al df general
    train_df = train_df.append(df_train_pivot, sort=False)
    test_df = test_df.append(df_test_pivot, sort=False)

#Reseteamos los indices
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)    

In [ ]:
#Verifiquemos los NaNs
list_df = [train_df,test_df]
for i_df in list_df:
    # Cantidad de valores nulos ordenados descendentemente
    total = i_df.isnull().sum().sort_values(ascending=False)
    # Porcetaje de lo que representa para cada columna
    percent = (i_df.isnull().sum()/i_df.isnull().count()).sort_values(ascending=False)
    # Mostramos los 2 resultados en conjunto.
    missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    print(missing_data.head(6))

Vemos que la columas de los sensores 22 y 23 no contienen informacion.
Por lo que vamos a eliminarlas

In [ ]:
#Eliminamos columnas con NaN
train_df = train_df.drop(columns=['s22','s23'])
test_df = test_df.drop(columns=['s22','s23'])

In [ ]:
test_df

Vamos a definir una duracion minima que deban tener las series para analizar.

Cualquiera que tenga un valor menor, la eliminaremos.

In [ ]:
# Duracion de las series temporales
lim_t = 50

In [ ]:
# Ahora debemos agregar una colunma al train que nos diga el tiempo que falta para la falla, 
# es decir, el tiempo restante hasta que cada turbina alcance valor maximo en la columa 'Time'.

# Definimos la columa RUL (Rest Util Life) y la completamos con NaN
train_df['RUL'] = np.nan

# Definimos listas vacias auxiliares
list_RUL_train = []
list_time_train = []
indx_delet = []

# Para cada unidad de cada archivo primero verificaremos su duracion.
# En caso de ser mayor a la duracion minima, guardaremos el tiempo total de la serie temporal.

for i_names in names:
    # Por archivo
    df_pivot1 = train_df[train_df['Name']==i_names]
    # Lista de unidade
    list_units = df_pivot1['Unit'].unique()
    for i_unit in list_units:
        # Informacion de la unidad
        df_pivot2 = df_pivot1[df_pivot1['Unit']==i_unit]
        # Duracion de la serie temporal
        len_ts = df_pivot2.shape[0]
      
        if len_ts<lim_t:
          #DURACION INCORRECTA  
          # En caso de ser muy corta, guardamos los indices para luego eliminarlos
          indx = df_pivot2.index
          indx_delet = np.concatenate((np.asarray(indx_delet),indx.values))
        else: 
          #DURACION CORRECTA
          # Tiempo maximo de cada unidad
          max_RUL = df_pivot2.Time.max()
          list_RUL_train.append(max_RUL)
          list_time_train.append(len_ts)
          # Indices
          indx = df_pivot2.index

          #Reemplazamos RUL por la diferencia entre el maximo y el tiempo de ciclo
          train_df.iloc[indx,-1] = (max_RUL - df_pivot2.Time).values

# Convertimos las listas en numpy arrays
list_time_train = np.asarray(list_time_train)
list_RUL_train = np.asarray(list_RUL_train)

# Eliminamos los indices que tenian duraciones menores
train_df.drop(index = indx_delet,inplace=True)

In [ ]:
display(train_df)

In [ ]:
plt.figure(figsize=(10,3))
sns.kdeplot(list_RUL_train, color= "Blue", shade = True)
plt.xlabel("RUL",size = 20)
plt.ylabel("Frecuencia",size = 20)
plt.title('Distribucion de RUL',size = 20)
plt.show()

In [ ]:
# Ahora para el test set, vamos a adjuntar el dato RUL de cada unidad al dataframe.

A diferencia de la informacion de entrenamiento, para la cual sabemos que la serie temporal termina **justo** en la falla, en los datos de test, nos dan una serie temporal y aparte tenemos los datos de cuanto le faltaba hasta la falla al equipo.

Esta informacion separada la vamos a consolidar.

In [ ]:
# Ahora para el test set, vamos a adjuntar el dato RUL de cada unidad al dataframe.
test_df['RUL'] = np.nan
list_RUL_test = []
list_time_test = []
indx_delet = []

for i_names in names:
    # Por archivo
    df_pivot1 = test_df[test_df['Name']==i_names]
    # lista de unidade
    list_units = df_pivot1['Unit'].unique()
    # lista de RUL
    df_RUL_pivot = pd.read_csv('RUL_'+i_names+'.txt', sep=" ", header=None)
    for i_unit in list_units:
        i_RUL = df_RUL_pivot.iloc[i_unit-1,0]
        # Por Unidad
        df_pivot2 = df_pivot1[df_pivot1['Unit']==i_unit]
        len_ts = df_pivot2.shape[0]
        if len_ts<lim_t:
          indx = df_pivot2.index
          indx_delet = np.concatenate((np.asarray(indx_delet),indx.values))
        else:
          # Tiempo maximo de cada unidad
          i_RUL = df_RUL_pivot.iloc[i_unit-1,0]
          list_RUL_test.append(i_RUL)
          list_time_test.append(len_ts)
          # Indices
          indx = df_pivot2.index
          # Reemplazamos RUL por la diferencia entre el maximo y el tiempo de ciclo
          test_df.iloc[indx,-1] = (i_RUL + df_pivot2.Time - 1).values[::-1]
          # el comando [::-1] nos invirte el orden del resultado
# Convertimos las listas en numpy arrays
list_RUL_test = np.asarray(list_RUL_test)
list_time_test = np.asarray(list_time_test)

# Eliminamos los indices con duraciones menores
test_df.drop(index = indx_delet,inplace=True)

In [ ]:
display(test_df)

In [ ]:
plt.figure(figsize=(10,3))
sns.kdeplot(list_RUL_test, color= "Blue", shade = True)
plt.xlabel("RUL",size = 20)
plt.ylabel("Frecuency",size = 20)
plt.title('Distribucion de RUL',size = 20)
plt.show()

plt.figure(figsize=(10,3))
sns.kdeplot(list_time_test, color= "Blue", shade = True)
plt.xlabel("Time-Serie",size = 20)
plt.ylabel("Frecuency",size = 20)
plt.title('Distribucion de Time-Serie',size = 20)
plt.show()

In [ ]:
global_df = pd.concat([train_df, test_df], join="inner")

In [ ]:
global_df

Definiremos una nueva columna, que identifique inequivocamente la unidad a la que corresponde la serie temporal.

Para eso el comando .diff() calcula la diferencia entre el valor de la fila actual y la fila anterior.

Solo tendremos un valor distinto de 0 cuando haya un cambio de unidad.

Cualquier valor distinto de 0 lo pasaremos a binario (0 o 1).

Luego iremos completando la columna con una suma acumulada.

Como la mayoria son 0, lo que iremos obteniendo en la columa "Unidad_new" sera el numero del equipo independientemente del archivo


In [ ]:
# Diferencias
global_df['Unit_new'] = global_df['Unit'].diff()

# El primer registro no tiene ningun registro previo, por lo que se nos llena con un nan, el cual lo reemplazamos por 1
global_df['Unit_new'].fillna(value=1,inplace=True)

# Pasamos a booleano (solo 1 o 0) y luego a entero
global_df['Unit_new'] = global_df['Unit_new'].astype(bool).astype(int)

# Suma acumulada
global_df['Unit_new'] = global_df['Unit_new'].cumsum()

# Lista con el numero de unidad 'global' del equipo
list_units_global = global_df['Unit_new'].unique()

list_time_global = np.concatenate((list_time_train,list_time_test))
t_max = max(list_time_global)

In [ ]:
global_df

# Split

Una vez que tenemos toda la informacion consolidada, vamos conformar, de manera aleatoria, los set de entrenamiento, validacion y testeo.

In [ ]:
from sklearn.model_selection import train_test_split
# Proporcional nuevo set
t_size= 0.15
r_seed = 0

# Train - Test
list_units_train, list_units_test, list_time_train, list_time_test = train_test_split(list_units_global, list_time_global, test_size=t_size, random_state=r_seed)
# Train - Validation
list_units_train, list_units_val, list_time_train, list_time_val = train_test_split(list_units_train, list_time_train, test_size=t_size, random_state=r_seed)

n_unidades_train = np.shape(list_units_train)[0]
n_unidades_val = np.shape(list_units_val)[0]
n_unidades_test = np.shape(list_units_test)[0]
print(f'Train: {n_unidades_train} | Val: {n_unidades_val} | Test: {n_unidades_test}')

# Dataframe con las unidades destinadas a cada set
train_df = global_df[global_df.Unit_new.isin(list_units_train)].copy()
val_df = global_df[global_df.Unit_new.isin(list_units_val)].copy()
test_df = global_df[global_df.Unit_new.isin(list_units_test)].copy()

# AUTOSCALING

In [ ]:
from sklearn import preprocessing

# Tipo de escalado que vamos a usar
#scaler = preprocessing.StandardScaler()
scaler = preprocessing.MinMaxScaler()


# TRAIN
#Aqui obtenemos los valores
train = train_df[features_columns].values.astype(float)
# Usando el metodo "fit_transform" le indicamos al objeto "scaler" que calcule los parametros para escalar, y que nos devuelva
# los datos ya escalados
# Si queremos acceder a los parametros del objeto, existe el metodo get_params()
train_scaled = scaler.fit_transform(train)
# Los datos ya normalzados los copiamos en el dataframe correspondiente
train_df[features_columns] = train_scaled

# VALIDATION
val = val_df[features_columns].values.astype(float)
# Ahora solo usamos el metodo "transform()" ya que el objeto "scaler" tiene
# internamente los parametros para escalar
val_scaled = scaler.transform(val)
val_df[features_columns] = val_scaled

# TEST
test = test_df[features_columns].values.astype(float)
test_scaled = scaler.transform(test)
test_df[features_columns] = test_scaled

En este punto vamos a dejar de tener al informacion en Pandas Dataframe, y la vamos pasar a numpy arrays.

Los inputs del modelo debe ser de la siguientes dimensiones:

**[ N° Elementos , Longitud Temporal, Features ]**

Pre-alojaremos con ceros los arrays n-dimensionales de los set de entramiento, validacion y testeo.

Usaremos como longitud temporal a la maxima duracion de las series temporales en los datos.

Las series que tengan menores duraciones seran completadas con ceros.

In [ ]:
# Features
n_caracteristicas = len(features_columns)

x_train = np.zeros((n_unidades_train, t_max,n_caracteristicas))
y_train = np.zeros((n_unidades_train, t_max, 1),dtype=np.int)

x_val = np.zeros((n_unidades_val, t_max,n_caracteristicas))
y_val = np.zeros((n_unidades_val, t_max, 1),dtype=np.int)

x_test = np.zeros((n_unidades_test, t_max,n_caracteristicas))
y_test = np.zeros((n_unidades_test, t_max, 1),dtype=np.int)

#Dimensiones de los nd-arrays
print(np.shape(x_train),np.shape(y_train))
print(np.shape(x_val),np.shape(y_val))
print(np.shape(x_test),np.shape(y_test))

Ahora para cada unidad de cada uno de los tres Dataframes, copiaremos los valores en los nd-array creados en la celda anterior.

In [ ]:
#TRAIN
for i, i_unit in enumerate(list_units_train):
    # Features a X, RUL a Y
    x = train_df[train_df['Unit_new']==i_unit][features_columns].values
    y = train_df[train_df['Unit_new']==i_unit]['RUL'].values
    # Ajustamos la dimension de Y
    y = y[...,np.newaxis]
    # list_time_train nos da la posicion de fin de la serie temporal 
    x_train[i,0:list_time_train[i],:] = x
    y_train[i,0:list_time_train[i],:] = y
#VALIDATION
for i, i_unit in enumerate(list_units_val):
    # Features a X, RUL a Y
    x = val_df[val_df['Unit_new']==i_unit][features_columns].values
    y = val_df[val_df['Unit_new']==i_unit]['RUL'].values
    # Ajustamos la dimension de Y
    y = y[...,np.newaxis]
    x_val[i,0:list_time_val[i],:] = x
    y_val[i,0:list_time_val[i],:] = y
#TEST
for i, i_unit in enumerate(list_units_test):
    # Features a X, RUL a Y
    x = test_df[test_df['Unit_new']==i_unit][features_columns].values
    y = test_df[test_df['Unit_new']==i_unit]['RUL'].values
    # Ajustamos la dimension de Y
    y = y[...,np.newaxis]
    x_test[i,0:list_time_test[i],:] = x
    y_test[i,0:list_time_test[i],:] = y

# LABELS

Para este ejercicio vamos a definir estados/clases en el que cada unidad se puede encontrar:

Clase/ Estado:
*   0 (Estado Critico) : Si RUL < 50
*   1 (Mantenimiento Preventivo): Si RUL $i\in$ [50,100]
*   2 (Estado Normal): Si RUL > 100

In [ ]:
class0 = y_train < 50
class1 = np.logical_and(y_train<=100, y_train>=50)
class2 = y_train > 100
y_train[class0]= 0
y_train[class1]= 1
y_train[class2]= 2

class0 = y_val < 50
class1 = np.logical_and(y_val<=100, y_val>=50)
class2 = y_val > 100
y_val[class0]= 0
y_val[class1]= 1
y_val[class2]= 2

class0 = y_test < 50
class1 = np.logical_and(y_test<=100, y_test>=50)
class2 = y_test > 100
y_test[class0]= 0
y_test[class1]= 1
y_test[class2]= 2

# DATA WINDOWING

In [ ]:
import random

def data_generator(batch_size, X, Y,list_time,t):
    num_examples = len(X)
    examples = zip(X, Y, list_time)
    examples = sorted(examples, key = lambda x: X[0].shape[0])
    end = num_examples - batch_size + 1
    batches = [examples[i:i + batch_size] for i in range(0, end, batch_size)]

    random.shuffle(batches)

    while True:
      for batch in batches:
        x_b, y_b, list_time_b = zip(*batch)
        # Pasamos a array
        x_b = np.asarray(x_b).copy()
        y_b = np.asarray(y_b,dtype=int).copy()
        list_time_b = np.asarray(list_time_b,dtype=int).copy()
        
        # Generamos los array para enviar al modelo
        x_batch = np.zeros((np.shape(x_b)[0],t,np.shape(x_b)[2]))
        y_batch = np.zeros((np.shape(x_b)[0]),dtype=int)

        #Loop para cada muestra / unidad
        for index in np.arange(len(x_batch)):
          
          # Tiempo maximo de la serie temporal
          t_max = list_time_b[index]
          if t_max>t:
            # Elegimos aleatoreamente un segmento de la serie temporal
            t_view = np.random.randint(t,t_max)
          else:
            # Para evitar error cuando la serie temporal tiene la duracion minima
            # y no habria valores aleatorios en el intervalo en cuestion.
            t_view = t

          x = x_b[index,t_view - t : t_view]
          y = y_b[index,t_view - t : t_view]

          # Ultimo valor del segmento de la serie temporal, el estado de la unidad
          y = y[-1]
          x_batch[index] = x
          y_batch[index] = y
        yield x_batch, y_batch

In [ ]:
# Batch Sizes
b_size_train = 48 

# El tamaño del batch de val y test va a ser el tamaño del respectivo set.
b_size_val = n_unidades_val 
b_size_test = n_unidades_test

#Definimos los 3 generadores
train_ds = data_generator(b_size,x_train,y_train,list_time_train,lim_t)
val_ds = data_generator(b_size_val,x_val,y_val,list_time_val,lim_t)
test_ds = data_generator(b_size_test,x_test,y_test,list_time_test,lim_t)

Existen otros metodos para formar las ventanas temporales, que quizas requieren mas "complejidad" para armarlos, pero se obtiene mejores rendimientos al entrenar, como por ejemplo:

* DataGenerator (Keras) :

  https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly

* DataWindowing :

  https://www.tensorflow.org/tutorials/structured_data/time_series#data_windowing




# MODEL

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input,Dense,LSTM,GRU,Bidirectional
tf.random.set_seed(0)

Anteriormente vimos que podemos definir los modelos utilizando 'models.Sequential'.

Una forma mucho mas versatil es utilizar la 'Functional API'.


En este ejemplo una la misma entrada se pasa a dos capas distintas. Ademas, las salidas de las capas 'B' y 'C' son utilizadas como output del modelo.

```
# Input Layer
inputs = Input(shape=(DIMENSION DE X))

# Layer A
x_a = Dense()(inputs)

# Layer B
x_b = Dense()(inputs)

# Layer C = B + A
x_a_b = tf.keras.layers.concatenate([x_a, x_b])
x_c = Dense()(x_a_b)

# Output LayerS
output1 = Dense()(x_c)
output2 = Dense()(x_b)

model = keras.Model(
    inputs=[inputs],
    outputs=[output1,output2]
    )

```


### ARCHITECTURE

In [ ]:
tf.keras.backend.clear_session()

model_inputs = Input(shape=(lim_t,n_caracteristicas))

# 1° RNN Layer
o = Bidirectional(GRU(18, return_sequences=True))(model_inputs)
# cada salida de esta layer, debe pasarse a la segunda capa, por lo que usamos
# el argumento 'return_sequences=True'

# 2° RNN Layer
o = Bidirectional(GRU(12, return_sequences=False))(o)
# en esta segunda layer RNN, como solo vamos a clasificar usando al ultima
# salida, ingresamos 'return_sequences=False'

# Output Layer
model_outputs = Dense(3,activation='softmax')(o)

model = tf.keras.Model(inputs=model_inputs, outputs=model_outputs)

### SUMMARY AND PLOT

In [ ]:
model.summary()
tf.keras.utils.plot_model(model,show_shapes=True)

# LOSS AND OPTMIZER

In [ ]:
# Optimizador
lr = 0.0005
# clip norm es MUY importante cuando entrenamos RNN para evitar la explosion del
# gradiente, que 'rompe' los pesos aprendidos.
opt = tf.keras.optimizers.Adam(learning_rate=lr,clipnorm=3)
# Funcion de penalizacion
loss_func = tf.keras.losses.SparseCategoricalCrossentropy() 

model.compile(optimizer=opt, loss=loss_func,metrics=['accuracy'])

# TRAINING

In [ ]:
training = model.fit(train_ds, epochs=1000,
                steps_per_epoch=n_unidades_train//b_size,
                validation_data=val_ds,validation_steps=n_unidades_val//b_size_val)

En este punto ya tenemos el modelo entrenado y podemos ver como fue el progreso del entrenamiento.

In [ ]:
#Loss
loss_history = training.history['loss']
val_loss_hist = training.history['val_loss']
epochs = range(1, len(loss_history) + 1)
plt.plot(epochs, loss_history, 'r', label='Training loss')
plt.plot(epochs, val_loss_hist, 'b', label='Validation loss')
plt.title('Training and validation Loss')
plt.legend()
plt.show()

#Accuracy
acc_history = training.history['accuracy']
val_acc_hist = training.history['val_accuracy']
epochs = range(1, len(acc_history) + 1)
plt.plot(epochs, acc_history, 'r', label='Training Acc.')
plt.plot(epochs, val_acc_hist, 'b', label='Validation Acc.')
plt.title('Training and validation Acc.')
plt.legend()
plt.show()

El entrenamiento se ve muy "ruidoso", por lo que vamos a suavizarlo

In [ ]:
# Suavizar
def curva_suavizada(puntos, factor):
    # En esta variable iremos guardando puntos
    puntos_suavizados = []
    for punto in puntos:
        if puntos_suavizados:
            anterior = puntos_suavizados[-1]
            puntos_suavizados.append(anterior * factor + punto * (1 - factor))
        else:
            puntos_suavizados.append(punto)
    return puntos_suavizados

In [ ]:
# Factor de suavizado
factor = 0.90
plt.plot(curva_suavizada(loss_history, factor),color='blue',label='Train')
plt.plot(curva_suavizada(val_loss_hist, factor),color='red',label='Validation')
plt.legend()
plt.show()
plt.plot(curva_suavizada(acc_history, factor),color='blue',label='Train')
plt.plot(curva_suavizada(val_acc_hist, factor),color='red',label='Validation')
plt.legend()
plt.show()

# EVALUATION

Finalmente vamos a evaluar sobre el set de testeo.


In [ ]:
model.evaluate(test_ds,steps=n_unidades_test//b_size_test)

# PREDICTION

Guardaremos para cada serie temporal, el valor de verdadero (y_true) y el valor predicho (y_pred).

Como vimos en clases anteriores, la salida de nuestra red cuando clasificamos es un one-hot vector con las 'probabilidades' de cada muestra (ya que usamos la funcion **softmax** en la ultima capa).


In [ ]:
# Generamos una ventana como muestra
x_to_pred, y_true = next(test_ds)

In [ ]:
# Predecimos
y_hat = model.predict(x = x_to_pred) # Salida de la red (%)
y_pred = np.argmax(y_hat, axis=1) # Clase de la salida (Clase)

# CONFUSION MATRIX

In [ ]:
from sklearn.metrics import confusion_matrix

c_matrix = confusion_matrix(y_true, y_pred)
ax= plt.subplot()
sns.heatmap(c_matrix, annot=True, ax = ax)
ax.set_xlabel('Prediccion')
ax.set_ylabel('Verdadero')
ax.set_title('Matriz de Confusion')
ax.xaxis.set_ticklabels(['Critico', 'Preventivo','Normal'])
ax.yaxis.set_ticklabels(['Critico', 'Preventivo','Normal'])
plt.show()

# CONSULTAS

In [ ]:
##########################################
#------------ CONSULTAS -----------------#
##########################################

# PROPUESTAS:

- **FEATURE SELECTION**:

    * Hemos usado **TODOS** los sensores, y los 3 modos de operacion, pero no analizamos si agregan o no informacion.

- **AUTOSCALING**:
  * El escalado que hicimos fue MinMax por cada sensor.Comparar con normalizacion.

- **CROSS-VALIDATION**:
  * Hacer una validacion cruzada 5 veces. Hasta ahora, cuando hicimos Cross-Validation, usamos 'GridSearchCV' de la libreria Sk-learn. Las librerias de deep learning, **NO** tienen integrada esta funcionalidad. Programar

- **REGRESION**:

  * En vez de clasificar los estados de las turbinas, estimen el valor de RUL.

  Ayuda:

  ```
  Arquitectura:
  model_outputs = Dense(1)

  Loss:
  loss = tf.keras.losses.MSE
  ```







